# Prediction setup (defense)
This notebook will create a dataset the can be used for our model that predicts fantasy points for defense.

In [1]:
import numpy as np
import pandas as pd
#from datetime import datetime, timedelta
from functions import get_current_weekday, calculate_nfl_week, get_next_sunday, get_current_year
import sqlite3

In [2]:
day = get_current_weekday()

In [3]:
date_string = get_next_sunday(day)

In [4]:
week = calculate_nfl_week(date_string)

In [5]:
season = get_current_year()

# Defense
Pulling from the database 2023 data so we can get points scored by team defenses in each game and derive our L8 features. L8 stands for last eight games.<br>

For the first half of the season, we'll need to pull the data we used to train our model, as the last eight games every team has played will go back into last season until every team has played eight games this season.

In [6]:
#Dynamically creating strings for the fd and dk table names that we use in our query.
# fd_table_name = "fd_table_" + str(week) + "_24"
# dk_table_name = "dk_table_" + str(week) + "_24"

# Getting data from previous notebook
This is the fantasy points for every defense from the beginning of the previous season through last week.

In [7]:
def_model = pd.read_csv('def_points_update_' + str(season) + '_' + str(week) + '.csv')

In [8]:
def_model.head()

,season,week,game_id,defteam,FD_Pts,DK_Pts
0,2022,1,2022_01_BAL_NYJ,BAL,11.0,11.0
1,2022,1,2022_01_BAL_NYJ,NYJ,4.0,4.0
2,2022,1,2022_01_BUF_LA,BUF,17.0,17.0
3,2022,1,2022_01_BUF_LA,LA,9.0,9.0
4,2022,1,2022_01_CLE_CAR,CAR,1.0,1.0


In [9]:
##Defensive fantasy points for FanDuel and DraftKings
# Sacks = 1
# Opponent-fumbles recovered = 2
# Return touchdowns = 6
# Extra Point Return = 2
# Safeties = 2
# Blocked Punt/Kick = 2
# Interceptions made = 2
# 0 points allowed = 10
# 1-6 points allowed = 7
# 7-13 points allowed = 4
# 14-20 points allowed = 1
# 21-27 points allowed = 0
# 28-34 points allowed = -1
# 35+ points allowed = -4

In [10]:
#Almost no difference between FD and DK defense scoring, so we take the mean and have one target variable
def_model['fantasy_points'] = def_model[['FD_Pts', 'DK_Pts']].mean(axis=1)

In [11]:
def_model.drop(columns = ['FD_Pts', 'DK_Pts'], inplace = True)

In [12]:
def_model.head()

,season,week,game_id,defteam,fantasy_points
0,2022,1,2022_01_BAL_NYJ,BAL,11.0
1,2022,1,2022_01_BAL_NYJ,NYJ,4.0
2,2022,1,2022_01_BUF_LA,BUF,17.0
3,2022,1,2022_01_BUF_LA,LA,9.0
4,2022,1,2022_01_CLE_CAR,CAR,1.0


In [13]:
def_model['defteam'] = def_model['defteam'].replace({'LA' : 'LAR', 'JAC' : 'JAX'})

In [14]:
def_model.rename(columns = {'defteam' : 'team'}, inplace = True)

In [15]:
def_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266 entries, 0 to 1265
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   season          1266 non-null   int64  
 1   week            1266 non-null   int64  
 2   game_id         1266 non-null   object 
 3   team            1266 non-null   object 
 4   fantasy_points  1266 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 49.6+ KB


# Need opponents
In all our code refactoring we've discovered that we've somehow lost the opposing teams to the fantasy defenses. We can fix that right here rather than jumping back into that monstrous notebook that compiles defensive fantasy points.

In [16]:
#Weekly data last two seasons
weekly_df = pd.read_csv('weekly_data_' + str(season) + '_' + str(week) + '.csv')

In [17]:
weekly_opponents = weekly_df.groupby(['season', 'week', 'recent_team', 'opponent_team']).first().reset_index()

In [18]:
weekly_opponents = weekly_opponents[['season', 'week', 'recent_team', 'opponent_team']]

In [19]:
weekly_opponents.rename(columns = {'recent_team': 'team', 'opponent_team': 'opponent'}, inplace = True)

In [20]:
weekly_opponents['team'] = weekly_opponents['team'].replace({'LA' : 'LAR', 'JAC': 'JAX'})
weekly_opponents['opponent'] = weekly_opponents['opponent'].replace({'LA' : 'LAR', 'JAC': 'JAX'})

In [21]:
weekly_opponents = weekly_opponents[weekly_opponents['team'] != weekly_opponents['opponent']]

In [22]:
def_model = pd.merge(def_model, weekly_opponents, on = ['season', 'week', 'team'], how = 'left')

In [23]:
def_model

,season,week,game_id,team,fantasy_points,opponent
0,2022,1,2022_01_BAL_NYJ,BAL,11.0,NYJ
1,2022,1,2022_01_BAL_NYJ,NYJ,4.0,BAL
2,2022,1,2022_01_BUF_LA,BUF,17.0,LAR
3,2022,1,2022_01_BUF_LA,LAR,9.0,BUF
4,2022,1,2022_01_CLE_CAR,CAR,1.0,CLE
...,...,...,...,...,...,...
1261,2024,4,2024_04_SEA_DET,SEA,1.0,DET
1262,2024,4,2024_04_TEN_MIA,MIA,2.0,TEN
1263,2024,4,2024_04_TEN_MIA,TEN,10.0,MIA
1264,2024,4,2024_04_WAS_ARI,ARI,-2.0,WAS


In [24]:
def_model[def_model.isna().any(axis = 1)]

,season,week,game_id,team,fantasy_points,opponent


# Adding live rows
We need to add a row for each team so we can calculate the respective variables entering the upcoming week of games. This will incorporate the last eight games of last season for each team.

In [25]:
# List of all 32 NFL teams
nfl_teams = list(def_model['opponent'].unique())

In [26]:
# Create a new DataFrame for the upcoming week, to be filled with predictive data
new_rows = pd.DataFrame({
    'season': [season] * 32,
    'week': [week] * 32,
    'game_id': [None] * 32,
    'fantasy_points': [None] * 32,
    'opponent': nfl_teams
})

In [27]:
#pd.set_option('display.max_rows', None)

In [28]:
# Concatenate the new rows to the existing DataFrame
def_model = pd.concat([def_model, new_rows], ignore_index=True)

In [29]:
def_model.head()

,season,week,game_id,team,fantasy_points,opponent
0,2022,1,2022_01_BAL_NYJ,BAL,11.0,NYJ
1,2022,1,2022_01_BAL_NYJ,NYJ,4.0,BAL
2,2022,1,2022_01_BUF_LA,BUF,17.0,LAR
3,2022,1,2022_01_BUF_LA,LAR,9.0,BUF
4,2022,1,2022_01_CLE_CAR,CAR,1.0,CLE


# Getting last week's points
We'll store this in a CSV and use it to check last week's predictions.

In [30]:
def_points_last_week = def_model[(def_model['season'] == 2024) & (def_model['week'] == week - 1)]

In [31]:
def_points_last_week.to_csv('def_target_' + str(season) + '_' + str(week - 1) + '.csv')

# Sequential game numbers
This gives each game a number, grouped by team, so that we can calculate means for features over the last eight games.

In [32]:
def_model['game_num'] = def_model.groupby(['opponent', 'season']).cumcount() + 1

# Rolling mean functions
These functions calculate the mean of points allowed to fantasy defenses over the previous eight games. If there are less than eight previous games in the current season, we go back to the previous season.<br>

We use both equal rolling means and weighted rolling means, with more weight being placed on more recent games.<br>

These functions will produce a dataframe that indicates the fantasy points scored by opposing defenses against the team in the opponent column.

In [33]:
# Function to calculate equally weighted rolling mean
def calculate_equal_rolling_mean(group):
    # Calculate the equally weighted rolling mean of the last 8 games, excluding the current game
    group['rolling_mean_8'] = (
        group['fantasy_points'].shift().rolling(window=8, min_periods=1).mean()
    )
    return group

# Apply the rolling mean calculation using groupby
def_model = def_model.groupby('opponent', group_keys=False).apply(calculate_equal_rolling_mean)

In [34]:
# Linear and Exponential weights
# linear_weights = np.array([0.25, 0.20, 0.15, 0.13, 0.10, 0.08, 0.05, 0.04])
# exponential_weights = np.array([0.27, 0.23, 0.19, 0.15, 0.11, 0.09, 0.07, 0.05])

# Corrected Linear and Exponential weights
linear_weights = np.array([0.01, 0.03, 0.06, 0.10, 0.14, 0.18, 0.24, 0.28])
exponential_weights = np.array([0.015, 0.025, 0.04, 0.07, 0.10, 0.15, 0.25, 0.35])

In [35]:
# Function to calculate rolling means with different weights
def calculate_weighted_means(group):
    # Calculate the linear weighted rolling mean
    group['linear_rolling_mean_8'] = (
        group['fantasy_points'].shift().rolling(window=8, min_periods=1)
        .apply(lambda x: np.dot(x, linear_weights[-len(x):]), raw=False)
    )
    
    # Calculate the exponential weighted rolling mean
    group['exponential_rolling_mean_8'] = (
        group['fantasy_points'].shift().rolling(window=8, min_periods=1)
        .apply(lambda x: np.dot(x, exponential_weights[-len(x):]), raw=False)
    )
    return group

In [36]:
# Apply the calculations using groupby
def_model = def_model.groupby('opponent', group_keys=False).apply(calculate_weighted_means)

In [37]:
def_model.rename(columns = {'rolling_mean_8': 'opp_vs_def_L8',\
                             'linear_rolling_mean_8': 'opp_vs_def_L8_lin',\
                            'exponential_rolling_mean_8': 'opp_vs_def_L8_exp'}, inplace = True)

In [38]:
def_model.drop(columns = ['game_num', 'game_id', 'fantasy_points'], inplace = True)

In [39]:
def_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1298 entries, 0 to 1297
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season             1298 non-null   int64  
 1   week               1298 non-null   int64  
 2   team               1266 non-null   object 
 3   opponent           1298 non-null   object 
 4   opp_vs_def_L8      1266 non-null   float64
 5   opp_vs_def_L8_lin  1042 non-null   float64
 6   opp_vs_def_L8_exp  1042 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 113.4+ KB


In [40]:
def_model.tail(10)

,season,week,team,opponent,opp_vs_def_L8,opp_vs_def_L8_lin,opp_vs_def_L8_exp
1288,2024,5,NaN,TEN,12.000,12.00,10.820
1289,2024,5,NaN,NYG,8.250,7.50,6.610
1290,2024,5,NaN,PHI,8.750,9.32,9.260
1291,2024,5,NaN,DET,2.750,3.39,3.075
1292,2024,5,NaN,PIT,4.500,6.02,5.990
1293,2024,5,NaN,CIN,6.875,5.32,4.340
1294,2024,5,NaN,SF,5.250,5.77,5.425
1295,2024,5,NaN,CHI,7.250,7.94,7.410
1296,2024,5,NaN,TB,7.125,7.35,7.465
1297,2024,5,NaN,DAL,5.375,5.03,4.515


In [41]:
def_model = def_model[(def_model['season'] == 2024) & (def_model['week'] == week)]

In [42]:
def_model = def_model[['opponent', 'opp_vs_def_L8', 'opp_vs_def_L8_lin', 'opp_vs_def_L8_exp']]

At this point we have rolling L8 averages of fantasy points scored by defense against teams in the opponent column heading into the upcoming week.

In [43]:
def_model

,opponent,opp_vs_def_L8,opp_vs_def_L8_lin,opp_vs_def_L8_exp
1266,NYJ,8.500,6.37,6.545
1267,BAL,4.875,2.99,1.985
1268,LAR,6.125,6.56,6.400
1269,BUF,4.750,3.85,4.045
1270,CLE,11.250,11.57,10.745
1271,CAR,8.500,6.90,5.290
1272,SEA,4.625,5.91,5.915
1273,DEN,7.625,7.24,6.580
1274,MIN,7.875,6.87,6.415
1275,GB,2.750,4.21,4.535


In [44]:
#spread_df.head()

# Bringing in FanDuel and DraftKings player lists
We'll filter this for the defenses.

In [45]:
#Getting FanDuel and DraftKings player lists
# Establish the connection to the database
conn = sqlite3.connect('nfl_dfs.db')

# Define the table names based on the week you want to retrieve
fd_table_name = 'fd_table_' + str(week) + '_24'
dk_table_name = 'dk_table_' + str(week) + '_24'

# Retrieve the FanDuel table
fd_df_retrieved = pd.read_sql_query(f"SELECT * FROM {fd_table_name}", conn)

# Retrieve the DraftKings table
dk_df_retrieved = pd.read_sql_query(f"SELECT * FROM {dk_table_name}", conn)

# Close the connection
conn.close()


In [46]:
fanduel_df = fd_df_retrieved
draftkings_df = dk_df_retrieved

In [47]:
#Filtering our DK and FD tables for defenses only
fanduel_df = fanduel_df[fanduel_df['position'] == 'D'].reset_index(drop = True)
draftkings_df = draftkings_df[draftkings_df['position'] == 'D'].reset_index(drop = True)

In [48]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week
0,107566-12549,SF,D,5000,SF,ARI,1,Active,10-06-2024,5
1,107566-12550,SEA,D,4800,SEA,NYG,1,Active,10-06-2024,5
2,107566-12533,GB,D,4700,GB,LAR,0,Active,10-06-2024,5
3,107566-12527,CHI,D,4600,CHI,CAR,1,Active,10-06-2024,5
4,107566-12555,BAL,D,4500,BAL,CIN,0,Active,10-06-2024,5


In [49]:
draftkings_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week
0,36142714,SF,D,3700,SF,ARI,1,Active,10-06-2024,5
1,36142715,CHI,D,3600,CHI,CAR,1,Active,10-06-2024,5
2,36142716,SEA,D,3500,SEA,NYG,1,Active,10-06-2024,5
3,36142717,MIA,D,3300,MIA,NE,0,Active,10-06-2024,5
4,36142718,LV,D,3200,LV,DEN,0,Active,10-06-2024,5


In [50]:
def_model

,opponent,opp_vs_def_L8,opp_vs_def_L8_lin,opp_vs_def_L8_exp
1266,NYJ,8.500,6.37,6.545
1267,BAL,4.875,2.99,1.985
1268,LAR,6.125,6.56,6.400
1269,BUF,4.750,3.85,4.045
1270,CLE,11.250,11.57,10.745
1271,CAR,8.500,6.90,5.290
1272,SEA,4.625,5.91,5.915
1273,DEN,7.625,7.24,6.580
1274,MIN,7.875,6.87,6.415
1275,GB,2.750,4.21,4.535


In [51]:
#Merging the L8 variables to the tables
fanduel_df = pd.merge(fanduel_df, def_model, on = ['opponent'], how = 'left')
draftkings_df = pd.merge(draftkings_df, def_model, on = ['opponent'], how = 'left')

In [52]:
fanduel_df.drop(columns = ['status'], inplace = True)
draftkings_df.drop(columns = ['status'], inplace = True)

In [53]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,date,week,opp_vs_def_L8,opp_vs_def_L8_lin,opp_vs_def_L8_exp
0,107566-12549,SF,D,5000,SF,ARI,1,10-06-2024,5,5.000,4.87,5.145
1,107566-12550,SEA,D,4800,SEA,NYG,1,10-06-2024,5,8.250,7.50,6.610
2,107566-12533,GB,D,4700,GB,LAR,0,10-06-2024,5,6.125,6.56,6.400
3,107566-12527,CHI,D,4600,CHI,CAR,1,10-06-2024,5,8.500,6.90,5.290
4,107566-12555,BAL,D,4500,BAL,CIN,0,10-06-2024,5,6.875,5.32,4.340


In [54]:
draftkings_df.head()

,ID,name,position,salary,team,opponent,home_team,date,week,opp_vs_def_L8,opp_vs_def_L8_lin,opp_vs_def_L8_exp
0,36142714,SF,D,3700,SF,ARI,1,10-06-2024,5,5.000,4.87,5.145
1,36142715,CHI,D,3600,CHI,CAR,1,10-06-2024,5,8.500,6.90,5.290
2,36142716,SEA,D,3500,SEA,NYG,1,10-06-2024,5,8.250,7.50,6.610
3,36142717,MIA,D,3300,MIA,NE,0,10-06-2024,5,11.625,14.40,14.970
4,36142718,LV,D,3200,LV,DEN,0,10-06-2024,5,7.625,7.24,6.580


# Odds, totals, outdoors and grass variables
Our odds CSVs will bring in the rest of the variables.

In [55]:
fd_spreads = pd.read_csv('fd_spreads_' + str(season) + '_' + str(week) + '.csv')
dk_spreads = pd.read_csv('dk_spreads_' + str(season) + '_' + str(week) + '.csv')

In [56]:
fd_spreads.head()

,Unnamed: 0,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,0,MIN,NYJ,40.5,0,1,0,-2.5,19.00,21.50,0,2024,5
1,1,CIN,BAL,48.5,1,0,0,2.5,25.50,23.00,12,2024,5
2,2,HOU,BUF,47.5,0,0,0,-1.5,23.00,24.50,0,2024,5
3,3,CHI,CAR,41.5,1,1,0,-3.5,19.00,22.50,16,2024,5
4,4,WAS,CLE,43.5,1,1,0,-3.0,20.25,23.25,5,2024,5


In [57]:
dk_spreads.head()

,Unnamed: 0,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,0,MIN,NYJ,40.5,0,1,0,-2.5,19.00,21.50,0,2024,5
1,1,CIN,BAL,48.5,1,0,0,2.5,25.50,23.00,12,2024,5
2,2,HOU,BUF,47.0,0,0,0,-1.0,23.00,24.00,0,2024,5
3,3,CHI,CAR,41.0,1,1,0,-4.0,18.50,22.50,16,2024,5
4,4,WAS,CLE,43.5,1,1,0,-3.0,20.25,23.25,5,2024,5


In [58]:
fanduel_df = pd.merge(fanduel_df, fd_spreads, on = ['team', 'opponent', 'week', 'home_team'], how = 'left')
draftkings_df = pd.merge(draftkings_df, dk_spreads, on = ['team', 'opponent', 'week', 'home_team'], how = 'left')

In [59]:
fanduel_df.drop(columns = ['Unnamed: 0'], inplace = True)
draftkings_df.drop(columns = ['Unnamed: 0'], inplace = True)

In [60]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,date,week,opp_vs_def_L8,opp_vs_def_L8_lin,opp_vs_def_L8_exp,total_line,outdoors,grass,spread_line,pred_total,opp_total,wind,season
0,107566-12549,SF,D,5000,SF,ARI,1,10-06-2024,5,5.000,4.87,5.145,49.5,1,1,7.5,28.50,21.00,6,2024
1,107566-12550,SEA,D,4800,SEA,NYG,1,10-06-2024,5,8.250,7.50,6.610,42.5,1,0,6.5,24.50,18.00,2,2024
2,107566-12533,GB,D,4700,GB,LAR,0,10-06-2024,5,6.125,6.56,6.400,48.5,0,0,3.0,25.75,22.75,0,2024
3,107566-12527,CHI,D,4600,CHI,CAR,1,10-06-2024,5,8.500,6.90,5.290,41.5,1,1,3.5,22.50,19.00,16,2024
4,107566-12555,BAL,D,4500,BAL,CIN,0,10-06-2024,5,6.875,5.32,4.340,48.5,1,0,2.5,25.50,23.00,12,2024


In [61]:
draftkings_df.head()

,ID,name,position,salary,team,opponent,home_team,date,week,opp_vs_def_L8,opp_vs_def_L8_lin,opp_vs_def_L8_exp,total_line,outdoors,grass,spread_line,pred_total,opp_total,wind,season
0,36142714,SF,D,3700,SF,ARI,1,10-06-2024,5,5.000,4.87,5.145,48.5,1,1,7.0,27.75,20.75,6,2024
1,36142715,CHI,D,3600,CHI,CAR,1,10-06-2024,5,8.500,6.90,5.290,41.0,1,1,4.0,22.50,18.50,16,2024
2,36142716,SEA,D,3500,SEA,NYG,1,10-06-2024,5,8.250,7.50,6.610,42.5,1,0,7.0,24.75,17.75,2,2024
3,36142717,MIA,D,3300,MIA,NE,0,10-06-2024,5,11.625,14.40,14.970,35.5,1,0,1.0,18.25,17.25,4,2024
4,36142718,LV,D,3200,LV,DEN,0,10-06-2024,5,7.625,7.24,6.580,35.5,1,1,-3.0,16.25,19.25,4,2024


In [62]:
fanduel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 22 non-null     object 
 1   name               22 non-null     object 
 2   position           22 non-null     object 
 3   salary             22 non-null     int64  
 4   team               22 non-null     object 
 5   opponent           22 non-null     object 
 6   home_team          22 non-null     int64  
 7   date               22 non-null     object 
 8   week               22 non-null     int64  
 9   opp_vs_def_L8      22 non-null     float64
 10  opp_vs_def_L8_lin  22 non-null     float64
 11  opp_vs_def_L8_exp  22 non-null     float64
 12  total_line         22 non-null     float64
 13  outdoors           22 non-null     int64  
 14  grass              22 non-null     int64  
 15  spread_line        22 non-null     float64
 16  pred_total         22 non-nu

In [63]:
draftkings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 20 non-null     int64  
 1   name               20 non-null     object 
 2   position           20 non-null     object 
 3   salary             20 non-null     int64  
 4   team               20 non-null     object 
 5   opponent           20 non-null     object 
 6   home_team          20 non-null     int64  
 7   date               20 non-null     object 
 8   week               20 non-null     int64  
 9   opp_vs_def_L8      20 non-null     float64
 10  opp_vs_def_L8_lin  20 non-null     float64
 11  opp_vs_def_L8_exp  20 non-null     float64
 12  total_line         20 non-null     float64
 13  outdoors           20 non-null     int64  
 14  grass              20 non-null     int64  
 15  spread_line        20 non-null     float64
 16  pred_total         20 non-nu

In [64]:
fanduel_df.drop(columns = ['ID', 'name', 'pred_total', 'wind'], inplace = True)
draftkings_df.drop(columns = ['ID', 'name', 'pred_total', 'wind'], inplace = True)

In [65]:
#Setting indexes so that the only columns are the ones that correspond with variables for the model
fanduel_df.set_index(['position', 'salary', 'team', 'opponent', 'date', 'week', 'season'], inplace = True)
draftkings_df.set_index(['position', 'salary', 'team', 'opponent', 'date', 'week', 'season'], inplace = True)

In [66]:
fanduel_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 22 entries, ('D', 5000, 'SF', 'ARI', '10-06-2024', 5, 2024) to ('D', 3000, 'NYG', 'SEA', '10-06-2024', 5, 2024)
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   home_team          22 non-null     int64  
 1   opp_vs_def_L8      22 non-null     float64
 2   opp_vs_def_L8_lin  22 non-null     float64
 3   opp_vs_def_L8_exp  22 non-null     float64
 4   total_line         22 non-null     float64
 5   outdoors           22 non-null     int64  
 6   grass              22 non-null     int64  
 7   spread_line        22 non-null     float64
 8   opp_total          22 non-null     float64
dtypes: float64(6), int64(3)
memory usage: 4.7+ KB


In [67]:
draftkings_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 20 entries, ('D', 3700, 'SF', 'ARI', '10-06-2024', 5, 2024) to ('D', 2300, 'ARI', 'SF', '10-06-2024', 5, 2024)
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   home_team          20 non-null     int64  
 1   opp_vs_def_L8      20 non-null     float64
 2   opp_vs_def_L8_lin  20 non-null     float64
 3   opp_vs_def_L8_exp  20 non-null     float64
 4   total_line         20 non-null     float64
 5   outdoors           20 non-null     int64  
 6   grass              20 non-null     int64  
 7   spread_line        20 non-null     float64
 8   opp_total          20 non-null     float64
dtypes: float64(6), int64(3)
memory usage: 4.4+ KB


In [68]:
#Now we need to set the variables in the order that they're trained in the model
fanduel_df = fanduel_df[['spread_line', 'home_team', 'outdoors', 'grass', 'total_line', 'opp_total', 'opp_vs_def_L8', 'opp_vs_def_L8_lin', 'opp_vs_def_L8_exp']]

In [69]:
draftkings_df = draftkings_df[['spread_line', 'home_team', 'outdoors', 'grass', 'total_line', 'opp_total', 'opp_vs_def_L8', 'opp_vs_def_L8_lin', 'opp_vs_def_L8_exp']]

In [70]:
fanduel_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 22 entries, ('D', 5000, 'SF', 'ARI', '10-06-2024', 5, 2024) to ('D', 3000, 'NYG', 'SEA', '10-06-2024', 5, 2024)
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   spread_line        22 non-null     float64
 1   home_team          22 non-null     int64  
 2   outdoors           22 non-null     int64  
 3   grass              22 non-null     int64  
 4   total_line         22 non-null     float64
 5   opp_total          22 non-null     float64
 6   opp_vs_def_L8      22 non-null     float64
 7   opp_vs_def_L8_lin  22 non-null     float64
 8   opp_vs_def_L8_exp  22 non-null     float64
dtypes: float64(6), int64(3)
memory usage: 4.7+ KB


In [71]:
draftkings_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 20 entries, ('D', 3700, 'SF', 'ARI', '10-06-2024', 5, 2024) to ('D', 2300, 'ARI', 'SF', '10-06-2024', 5, 2024)
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   spread_line        20 non-null     float64
 1   home_team          20 non-null     int64  
 2   outdoors           20 non-null     int64  
 3   grass              20 non-null     int64  
 4   total_line         20 non-null     float64
 5   opp_total          20 non-null     float64
 6   opp_vs_def_L8      20 non-null     float64
 7   opp_vs_def_L8_lin  20 non-null     float64
 8   opp_vs_def_L8_exp  20 non-null     float64
dtypes: float64(6), int64(3)
memory usage: 4.4+ KB


In [72]:
fanduel_df.to_csv('FD_def_for_model_' + str(season) + '_' + str(week) + '.csv')

In [73]:
draftkings_df.to_csv('DK_def_for_model_' + str(season) + '_' + str(week) + '.csv')